# Stock Predictor - Tutorial Completo

Este notebook muestra cómo usar el sistema de predicción de acciones paso a paso.

## 1. Importar librerías necesarias

In [ ]:
import sys
sys.path.append('..')  # Añadir directorio padre al path

from data_loader import StockDataLoader
from preprocessing import StockPreprocessor
from train import StockTrainer
from predict import StockPredictor
from config import Config, get_quick_test_config

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Configurar matplotlib
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (15, 8)

## 2. Explorar y descargar datos

In [ ]:
# Crear cargador de datos
loader = StockDataLoader(data_dir="../data")

# Descargar datos de Apple
ticker = "AAPL"
data = loader.download_stock_data(ticker, period="2y")

print(f"\nDatos descargados: {len(data)} días")
print(f"\nPrimeras filas:")
data.head()

In [ ]:
# Visualizar los datos
plt.figure(figsize=(15, 6))
plt.plot(data['Date'], data['Close'], linewidth=2)
plt.title(f'Precio de cierre histórico de {ticker}', fontsize=16, fontweight='bold')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Precio ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
data[['Open', 'High', 'Low', 'Close', 'Volume']].describe()

## 3. Preprocesar datos

In [ ]:
# Crear preprocesador
preprocessor = StockPreprocessor(sequence_length=60)

# Preparar características
processed_data = preprocessor.prepare_features(data)

print(f"Datos procesados: {len(processed_data)} días")
print(f"Características: {len(preprocessor.feature_columns)}")
print(f"\nCaracterísticas incluidas:")
for i, col in enumerate(preprocessor.feature_columns, 1):
    print(f"  {i}. {col}")

In [ ]:
# Visualizar algunos indicadores técnicos
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Precio con Moving Averages
axes[0].plot(processed_data['Date'], processed_data['Close'], label='Close', linewidth=2)
axes[0].plot(processed_data['Date'], processed_data['MA_7'], label='MA-7', alpha=0.7)
axes[0].plot(processed_data['Date'], processed_data['MA_21'], label='MA-21', alpha=0.7)
axes[0].plot(processed_data['Date'], processed_data['MA_50'], label='MA-50', alpha=0.7)
axes[0].set_title('Precio con Moving Averages', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Precio ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(processed_data['Date'], processed_data['RSI'], linewidth=2, color='purple')
axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.5, label='Sobrecompra')
axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.5, label='Sobreventa')
axes[1].set_title('RSI (Relative Strength Index)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('RSI')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Bollinger Bands
axes[2].plot(processed_data['Date'], processed_data['Close'], label='Close', linewidth=2)
axes[2].plot(processed_data['Date'], processed_data['BB_Upper'], label='BB Upper', alpha=0.5, linestyle='--')
axes[2].plot(processed_data['Date'], processed_data['BB_Middle'], label='BB Middle', alpha=0.5)
axes[2].plot(processed_data['Date'], processed_data['BB_Lower'], label='BB Lower', alpha=0.5, linestyle='--')
axes[2].fill_between(processed_data['Date'], processed_data['BB_Upper'], processed_data['BB_Lower'], alpha=0.1)
axes[2].set_title('Bollinger Bands', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Fecha')
axes[2].set_ylabel('Precio ($)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Entrenar el modelo

In [ ]:
# Usar configuración para pruebas rápidas
config = get_quick_test_config()
config.data.ticker = "AAPL"
config.print_config()

In [ ]:
# Crear entrenador
trainer = StockTrainer(
    model_type="lstm",
    hidden_size=128,
    num_layers=2,
    dropout=0.2,
    learning_rate=0.001,
    batch_size=32,
    sequence_length=60
)

# Preparar datos
train_loader, val_loader = trainer.prepare_data(
    ticker="AAPL",
    period="2y",
    train_ratio=0.8
)

In [ ]:
# Entrenar (esto puede tomar varios minutos)
history = trainer.train(
    train_loader,
    val_loader,
    epochs=20,  # Reducido para el notebook
    patience=5,
    save_dir="../models"
)

# Guardar preprocesador
trainer.preprocessor.save_scaler("../models/scaler.pkl")

In [ ]:
# Visualizar historial de entrenamiento
trainer.plot_training_history(save_path="../logs/training_history.png")

## 5. Hacer predicciones

In [ ]:
# Cargar predictor
predictor = StockPredictor(
    model_path="../models/best_model.pth",
    scaler_path="../models/scaler.pkl"
)

In [ ]:
# Predecir el próximo día
predicted_price, data = predictor.predict_next_day("AAPL")

In [ ]:
# Predecir múltiples días
predictor.plot_prediction(
    ticker="AAPL",
    days_ahead=7,
    period="3mo",
    save_path="../logs/prediction.png"
)

## 6. Backtesting

In [ ]:
# Realizar backtesting
metrics = predictor.backtest(
    ticker="AAPL",
    test_days=30,
    period="2y"
)

print("\nMétricas detalladas:")
print(f"MSE: {metrics['mse']:.6f}")
print(f"RMSE: ${metrics['rmse']:.2f}")
print(f"MAE: ${metrics['mae']:.2f}")
print(f"MAPE: {metrics['mape']:.2f}%")
print(f"Precisión de dirección: {metrics['direction_accuracy']:.2f}%")

In [ ]:
# Visualizar backtesting
predictor.plot_backtest(
    ticker="AAPL",
    test_days=30,
    save_path="../logs/backtest.png"
)

## 7. Experimentar con diferentes acciones

In [ ]:
# Probar con diferentes acciones
tickers = ["AAPL", "GOOGL", "MSFT", "TSLA"]

for ticker in tickers:
    try:
        print(f"\n{'='*60}")
        print(f"Analizando {ticker}")
        print(f"{'='*60}")
        
        predicted_price, _ = predictor.predict_next_day(ticker)
        
    except Exception as e:
        print(f"Error con {ticker}: {str(e)}")

## 8. Comparar diferentes modelos

In [ ]:
# Entrenar diferentes modelos y comparar
model_types = ["lstm", "gru"]
results = {}

for model_type in model_types:
    print(f"\n{'='*60}")
    print(f"Entrenando modelo {model_type.upper()}")
    print(f"{'='*60}")
    
    trainer = StockTrainer(
        model_type=model_type,
        hidden_size=128,
        num_layers=2,
        dropout=0.2,
        learning_rate=0.001,
        batch_size=32,
        sequence_length=60
    )
    
    train_loader, val_loader = trainer.prepare_data(
        ticker="AAPL",
        period="1y",
        train_ratio=0.8
    )
    
    history = trainer.train(
        train_loader,
        val_loader,
        epochs=10,
        patience=5,
        save_dir=f"../models/{model_type}"
    )
    
    results[model_type] = {
        'best_val_loss': history['best_val_loss'],
        'training_time': history['training_time']
    }

# Comparar resultados
print("\n" + "="*60)
print("COMPARACIÓN DE MODELOS")
print("="*60)
for model_type, metrics in results.items():
    print(f"\n{model_type.upper()}:")
    print(f"  Mejor val_loss: {metrics['best_val_loss']:.6f}")
    print(f"  RMSE: {np.sqrt(metrics['best_val_loss']):.6f}")
    print(f"  Tiempo de entrenamiento: {metrics['training_time']:.2f}s")

## Conclusiones

Este notebook ha demostrado:

1. Cómo descargar y visualizar datos del mercado
2. Cómo preprocesar datos y crear indicadores técnicos
3. Cómo entrenar un modelo de predicción
4. Cómo hacer predicciones operativas
5. Cómo evaluar el modelo mediante backtesting
6. Cómo comparar diferentes arquitecturas de modelos

**Recuerda**: Este proyecto es solo para fines educativos. No uses estos modelos para tomar decisiones de inversión reales sin consultar a profesionales financieros.